In [ ]:
import pickle
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from ydata_profiling import ProfileReport

pd.set_option('display.max_columns', None)
df = pd.read_csv('merge2018-tratado.csv')

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
#a media representa a proporção de valores 1, ou seja, valores ausentes.
df.isnull().mean()

In [ ]:
#seleciona todas as linhas onde a média de valores ausentes é maior que 0 e ordena
df.loc[:, df.isnull().mean() > 0].isnull().mean().sort_values

In [ ]:
#serializar a informação obtida anteriormente
#obter o index das colunas com pelo menos 17% dos valores ausentes e remover do banco
missing_proporcion = df.loc[:, df.isnull().mean() > 0].isnull().mean().sort_values()
cols_drop = missing_proporcion[missing_proporcion > 0.17].index
df = df.drop(columns=cols_drop)
df.shape

In [7]:
#seleciona as colunas do data set com tipo numerico e categorico e as separa
numerical_cols = df.select_dtypes(exclude=['object']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

In [8]:
numerical_df = df[numerical_cols]
categorical_df = df[categorical_cols]

In [ ]:
msno.matrix(numerical_df, labels=True)

In [ ]:
msno.matrix(categorical_df, labels=True)

In [ ]:
df['Situação no Curso'].value_counts()

In [ ]:
df['Situação no Curso'].value_counts().plot(kind='bar')
df[df['Situação no Curso'].isnull()].shape

In [ ]:
df = df[ df['Situação no Curso'].isin(['Evasão', 'Matriculado', 'Concluído', 'Cancelado', 'Formado', 'Jubilado', 'Trancado Voluntariamente', 'Trancado', 'Cancelamento por Desligamento', 'Transferido Interno', 'Transferido Externo', 'Intercâmbio'])]
df['class'] = df['Situação no Curso'].apply(lambda x:1 if x in ['Evasão', 'Cancelado', 'Jubilado', 'Cancelamento por Desligamento'] else 0)
df['class'].value_counts().plot(kind='bar')
df[df['class'].isnull()].shape

In [14]:
x = df.drop(['class', 'Situação no Curso', 'Situação no Período', 'Unnamed: 0'], axis=1)
y = df['class']

In [ ]:
# Recriar o DataFrame incluindo a coluna 'class'
df_modificado = x.copy()
df_modificado['class'] = df['class']

profile = ProfileReport(df_modificado, title="Relatório de Análise - Dados de Evasão", explorative=True)

profile.to_notebook_iframe()

profile.to_file("relatorio_dados_evasao_modificado.html")

# Exibir as contagens da variável alvo
print(df_modificado['class'].value_counts())

In [ ]:
msno.matrix(x, labels=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [18]:
# Recalculando colunas numéricas e categóricas após as remoções
numerical_cols = x_train.select_dtypes(exclude=['object']).columns
categorical_cols = x_train.select_dtypes(include=['object']).columns

In [19]:
#imputar missing, tratar e padronizar variaveis categoricas somente após definir treino e teste.
#os nomes nos pipelines e columnsTransformer são para facilitar a busca de erros

#imputa os valores missing nas variaveis numericas com a mediana
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])

#imputa os valores missing das variaveis categoricas com o valor mais frequente
#aplica o OneHotEncoder transformado as variaveies em dummies
categorical_transformer = Pipeline(steps=[
    ("imputer2", SimpleImputer(strategy='most_frequent')),
    ("enclude", OneHotEncoder(handle_unknown='ignore'))
])

#no columnsTransformer passamos um nome, o passo que queremos aplicar, nesse caso o que foi feito anteriormente pelo pipeline e as colunas que desejamos aplicar os passos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [ ]:
#utilizamos fit apenas no treino
#imputa e transforma os dados do treino
x_train_transformed = preprocessor.fit_transform(x_train)

#imputa e transforma os dados de teste
x_test_transformed = preprocessor.transform(x_test)

# Inicializar o SMOTE
smote = SMOTE(random_state=42)

# Aplicar o SMOTE apenas no conjunto de treino
x_train_balanced, y_train_balanced = smote.fit_resample(x_train_transformed, y_train)

# Verificar a nova distribuição
print(f'Distribuição de classes após SMOTE: \n{y_train_balanced.value_counts()}')

In [ ]:
#criamos o modelo e treinamos
rf = RandomForestClassifier()
rf.fit(x_train_balanced, y_train_balanced)

In [ ]:
# Obter a importância das características
importances = rf.feature_importances_

# Criar um DataFrame com as importâncias
feature_importance_df = pd.DataFrame({
    'Feature': numerical_cols.tolist() + list(preprocessor.named_transformers_['cat']['enclude'].get_feature_names_out(categorical_cols)),
    'Importance': importances
})

# Ordenar as características pela importância
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Visualizar as importâncias
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_importance_df.head(20))
plt.title('Top 20 Features by Importance')
plt.show()

In [ ]:
y_pred = rf.predict(x_test_transformed)
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

In [ ]:
from sklearn.metrics import roc_curve, auc

y_proba = rf.predict_proba(x_test_transformed)[:, 1]  # Probabilidades para a classe positiva
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend()
plt.show()

In [29]:
# Cria um arquivo para armazenar o modelo criado, em formato binário
rf_picle = open('modelo_pred.pickle', 'wb')

# Serializa o objeto rf (modelo criado), para que possa ser usado posteriormente
pickle.dump(rf, rf_picle)

# Fecha o arquivo para garantir que os dados sejam salvos
rf_picle.close()

# Obter os valores únicos da variável alvo após o balanceamento com SMOTE
class_uniques = y_train_balanced.unique()

# Abre um arquivo para salvar os valores únicos da variável alvo
output_pickle = open('output_class.pickle', 'wb')

# Serializa o objeto class_uniques, que são os valores da variável alvo, escrevendo-os dentro do arquivo output_pickle
pickle.dump(class_uniques, output_pickle)

# Fecha o arquivo garantindo que suas informações serão gravadas no disco
output_pickle.close()